In [39]:
import pandas as pd
import numpy as np
import bs4, requests, re, pickle

import nltk
from unicodedata import normalize
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import  cosine_similarity


In [40]:
df_gizmodo = pd.read_csv('aplicacao_modelagem_topicos/data/noticias_gizmodo.csv')
df_gizmodo.shape

(1988, 6)

In [41]:
df_gizmodo.head()

,autor,categoria,publicacao,texto,titulo,url
0,David Nield,smartphones,2018-11-25 17:11:58,O que você procura saber ao escolher um novo s...,5 detalhes para ficar de olho nas especificaçõ...,https://gizmodo.uol.com.br/5-especificacoes-sm...
1,Ryan F. Mandelbaum,espaço,2018-11-25 15:28:37,Cinco sextos da “coisa” que compõe o universo ...,O que é matéria escura e por que ninguém a enc...,https://gizmodo.uol.com.br/o-que-e-materia-esc...
2,George Dvorsky,animais,2018-11-25 12:59:49,"Como humanos, gostamos de pensar que somos a ú...",Chimpanzés podem ser manipuladores assim como ...,https://gizmodo.uol.com.br/chimpanzes-manipula...
3,Adam Clark Estes,review,2018-11-25 10:49:05,Vou começar dizendo algo importante sobre os h...,"[Review] Headphones Surface, da Microsoft, são...",https://gizmodo.uol.com.br/review-microsoft-he...
4,Leo Escudeiro,China,2018-11-24 17:27:39,"Em agosto, Donald Trump proibiu o uso de tecno...",Estados Unidos pedem que governos aliados não ...,https://gizmodo.uol.com.br/estados-unidos-gove...


In [42]:
df_gizmodo['doc'] = df_gizmodo.apply(lambda row:row['categoria'] +' '+ row['titulo'] +' '+  row['texto'], axis=1)
df_gizmodo['doc'].head()

0    smartphones 5 detalhes para ficar de olho nas ...
1    espaço O que é matéria escura e por que ningué...
2    animais Chimpanzés podem ser manipuladores ass...
3    review [Review] Headphones Surface, da Microso...
4    China Estados Unidos pedem que governos aliado...
Name: doc, dtype: object

In [43]:
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords = normalize('NFKD', ' '.join(stopwords)).encode('ASCII', 'ignore').decode().split()

stemmer = nltk.stem.RSLPStemmer()

def processar_texto(texto):
    if texto is None or not texto:
        return ''

    # Trasnformação do texto em minúsculo e remoção de termo
    texto = str(texto).lower()

    # Removendo acentuação
    texto = normalize('NFKD', texto).encode('ASCII', 'ignore').decode()

    # Removendo Pontuação, stopwords, palavras com número e aplicando stemming
    texto = ' '.join([stemmer.stem(c) for c in nltk.word_tokenize(texto) 
                      if (c not in punctuation) and (c not in stopwords) and not (re.match('.*[\d_].*', c)) ])

    return texto


In [44]:
df_gizmodo

,autor,categoria,publicacao,texto,titulo,url,doc
0,David Nield,smartphones,2018-11-25 17:11:58,O que você procura saber ao escolher um novo s...,5 detalhes para ficar de olho nas especificaçõ...,https://gizmodo.uol.com.br/5-especificacoes-sm...,smartphones 5 detalhes para ficar de olho nas ...
1,Ryan F. Mandelbaum,espaço,2018-11-25 15:28:37,Cinco sextos da “coisa” que compõe o universo ...,O que é matéria escura e por que ninguém a enc...,https://gizmodo.uol.com.br/o-que-e-materia-esc...,espaço O que é matéria escura e por que ningué...
2,George Dvorsky,animais,2018-11-25 12:59:49,"Como humanos, gostamos de pensar que somos a ú...",Chimpanzés podem ser manipuladores assim como ...,https://gizmodo.uol.com.br/chimpanzes-manipula...,animais Chimpanzés podem ser manipuladores ass...
3,Adam Clark Estes,review,2018-11-25 10:49:05,Vou começar dizendo algo importante sobre os h...,"[Review] Headphones Surface, da Microsoft, são...",https://gizmodo.uol.com.br/review-microsoft-he...,"review [Review] Headphones Surface, da Microso..."
4,Leo Escudeiro,China,2018-11-24 17:27:39,"Em agosto, Donald Trump proibiu o uso de tecno...",Estados Unidos pedem que governos aliados não ...,https://gizmodo.uol.com.br/estados-unidos-gove...,China Estados Unidos pedem que governos aliado...
5,George Dvorsky,Vídeo,2018-11-24 14:58:06,"O astronauta Alexander Gerst, da Agência Espac...",Ver o lançamento de um foguete do ponto de vis...,https://gizmodo.uol.com.br/video-lancamento-so...,Vídeo Ver o lançamento de um foguete do ponto ...
6,Giovanni Santa Rosa,Apps da semana,2018-11-24 13:03:04,Apps para usar um DNS mais seguro e acompanhar...,Os melhores apps da semana para Android,https://gizmodo.uol.com.br/melhores-apps-andro...,Apps da semana Os melhores apps da semana para...
7,Alessandro Feitosa Jr.,Apps da semana,2018-11-24 11:04:19,"Apps para ver cotações de criptomoedas, deixar...",Os melhores apps da semana para iPad e iPhone,https://gizmodo.uol.com.br/melhores-apps-ipad-...,Apps da semana Os melhores apps da semana para...
8,Alessandro Feitosa Jr.,Facebook,2018-11-23 18:07:41,Um comitê global de parlamentares vai sabatina...,Brasil fará parte de comitê internacional que ...,https://gizmodo.uol.com.br/facebook-comite-int...,Facebook Brasil fará parte de comitê internaci...
9,Giovanni Santa Rosa,Processadores,2018-11-23 17:49:15,"A Qualcomm, uma das mais importantes fabricant...",Qualcomm deve anunciar seu novo processador pa...,https://gizmodo.uol.com.br/qualcomm-novo-proce...,Processadores Qualcomm deve anunciar seu novo ...


In [45]:
df_gizmodo['doc2'] = df_gizmodo['doc'].apply(processar_texto)
df_gizmodo[['doc', 'doc2']].head(15)

,doc,doc2
0,smartphones 5 detalhes para ficar de olho nas ...,smartphon detalh fic olh especificaco prox sma...
1,espaço O que é matéria escura e por que ningué...,espac mat escur ning encontr aind cinc sext co...
2,animais Chimpanzés podem ser manipuladores ass...,animal chimpanz pod ser manipul assim human di...
3,"review [Review] Headphones Surface, da Microso...",review review headphon surfac microsoft pouc d...
4,China Estados Unidos pedem que governos aliado...,chin est unid ped govern ali us disposi chin h...
5,Vídeo Ver o lançamento de um foguete do ponto ...,vide ver lanc foguet pont vist espac parec coi...
6,Apps da semana Os melhores apps da semana para...,app seman melhor app seman android app us dm s...
7,Apps da semana Os melhores apps da semana para...,app seman melhor app seman ipad iphon app ver ...
8,Facebook Brasil fará parte de comitê internaci...,facebook brasil far part comit internac questi...
9,Processadores Qualcomm deve anunciar seu novo ...,process qualcomm dev anunci nov process aparel...


In [46]:
df_gizmodo['doc'] = df_gizmodo['doc2']
df_gizmodo = df_gizmodo.drop(['doc2'], axis=1)

In [47]:
vectorizer = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1, 1),
    max_features=None,
    binary=False,
    use_idf=True
)

tfidf_matrix = vectorizer.fit_transform(df_gizmodo['doc'])

In [48]:
print(vectorizer.get_feature_names()[:100])

['aaa', 'aaap', 'aalen', 'aalt', 'aaltouniversityac', 'aand', 'aap', 'aarhu', 'aaron', 'ab', 'aba', 'abac', 'abacat', 'abacu', 'abaf', 'abaix', 'abaixo', 'abajur', 'abal', 'abaliz', 'aban', 'abandon', 'abandona', 'abarc', 'abarqu', 'abarrot', 'abast', 'abastec', 'abat', 'abaul', 'abbey', 'abc', 'abcd', 'abckidtv', 'abcomm', 'abdic', 'abdicaca', 'abdom', 'abdul', 'abe', 'abel', 'abelh', 'abell', 'abenco', 'abert', 'abijeet', 'abili', 'abism', 'abloy', 'abnous', 'aboard', 'abob', 'abocanh', 'abog', 'abol', 'abolica', 'abomin', 'abord', 'aborigen', 'aborrec', 'abort', 'abou', 'about', 'aboutskin', 'abr', 'abra', 'abraa', 'abrac', 'abraca', 'abrah', 'abrahosting', 'abrand', 'abrang', 'abrangenc', 'abrevi', 'abreviaca', 'abri', 'abrig', 'abril', 'abrupt', 'absolut', 'absolv', 'absorca', 'absorco', 'absorv', 'abstera', 'abstev', 'abstinenc', 'abstiv', 'abstraco', 'abstrat', 'absurd', 'abt', 'abu', 'abund', 'abundanc', 'abus', 'abusara', 'abutr', 'ac']


In [49]:
words = pd.DataFrame(tfidf_matrix.todense(), columns=vectorizer.get_feature_names())

In [50]:
words.head()

,aaa,aaap,aalen,aalt,aaltouniversityac,aand,aap,aarhu,aaron,ab,...,zui,zul,zumb,zumbif,zumbific,zumvel,zurbuchen,zuriqu,zwicky,zykov
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015552,0.035184,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.029805,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0


In [51]:
def obter_noticias_similares(df_noticias, df_words, texto_pesquisa=''):
    texto = processar_texto(texto_pesquisa)
    df_texto = pd.DataFrame({'doc': [texto]})
    
    df_words_pesquisa = pd.DataFrame(
                            vectorizer.transform(df_texto['doc']).todense(),
                            columns=vectorizer.get_feature_names()
                        )
    
    df_words_pesquisa = df_words.append(df_words_pesquisa, ignore_index=True, sort=False)
    
    # Obtém a similaridade das palavras
    sim = cosine_similarity(df_words_pesquisa)
    
    
    # Obtém as similaridades dá última coluna, referente ao texto pesquisado
    coluna_pesquisa = pd.Series(sim[-1])
    
    return df_noticias.loc[ coluna_pesquisa.sort_values(ascending=False)[1:].index ].drop(['doc'], axis=1)

In [52]:
df_similares = obter_noticias_similares(
                            df_noticias=df_gizmodo, 
                            df_words=words, 
                            texto_pesquisa='extraterrestres nasa'
                )

In [53]:
df_similares.head()['url']

974     https://gizmodo.uol.com.br/tess-nasa-busca-pla...
1970    https://gizmodo.uol.com.br/nasa-atrasa-missao-...
15          https://gizmodo.uol.com.br/nasa-teste-spacex/
1155    https://gizmodo.uol.com.br/nasa-kepler-pouco-c...
801     https://gizmodo.uol.com.br/nasa-opportunity-te...
Name: url, dtype: object

In [54]:
with open('aplicacao_modelagem_topicos/tfidf_vectorizer', 'wb') as f:
    pickle.dump(vectorizer, file=f)
    
words.to_csv('aplicacao_modelagem_topicos/data/df_words_tfidf.csv', index=False)